# Movielens 영화 추천
## 코드

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

### 1) 데이터 준비와 전처리

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/data/recommend_data/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')
# 3점이상인 데이터가 전제데이터의 83.63%를 차지한다.

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다. 별점 -> 시청횟수
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings.tail()

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648
1000208,6040,1097,4,956715569


In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/data/recommend_data/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
print(temp_user_data)

if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

### 2) 분석

- **ratings에 있는 유니크한 영화 개수**

In [6]:
ratings['movie_id'].nunique()

3628

- **rating에 있는 유니크한 사용자 수**

In [7]:
ratings['user_id'].nunique()

6039

- **가장 인기 있는 영화 30개(인기순)**

In [8]:
# ratings의 movie_id로 그룹화하여, rating을 합산한 후 상위 30개를 뽑아온다.
rating_count = ratings.groupby('movie_id')['count'].count()
print(len(rating_count))  # groupby 개수 == 유니크한 영화 개수
rating_count.sort_values(ascending=False).head(30)

3628


movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: count, dtype: int64

### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가

In [9]:
# ratings : user_id, movie_id, count, timestamp
# movies : movie_id, title, genre

# 내가 선호하는 영화가 movies에 있는지 체크
def check_movie(my_movies, movies=movies):
    return [True if (movies['title']==x).any() else False for x in my_movies]

my_movies_title = ['Dead Poets Society (1989)', 'Good Will Hunting (1997)', 'Toy Story (1995)', 'Notting Hill (1999)', 'Forrest Gump (1994)']
print(check_movie(my_movies_title, movies))

# 영화리스트를 index로 변환해주는 함수
def title2index(my_movies, movies):
    return [movies[movies['title'] == movie]['movie_id'].values[0] for movie in my_movies]

my_movies_index = title2index(my_movies_title, movies)
print(my_movies_index)

# 사용자 인덱스 생성
my_id = ratings['user_id'].max() + 1

# pandas dataframe으로 바꿔줌
my_movie_list = pd.DataFrame({'user_id': [my_id]*5, 'movie_id': my_movies_index, 'count': [5.0]*5, 'timestamp': [956715648]*5})

# ratings에 추가
if not ratings.isin({'user_id':[my_id]})['user_id'].any():
    ratings = ratings.append(my_movie_list)

ratings.tail(10)

[True, True, True, True, True]
[1246, 1704, 1, 2671, 356]


,user_id,movie_id,count,timestamp
1000203,6040,1090,3.0,956715518
1000205,6040,1094,5.0,956704887
1000206,6040,562,5.0,956704746
1000207,6040,1096,4.0,956715648
1000208,6040,1097,4.0,956715569
0,6041,1246,5.0,956715648
1,6041,1704,5.0,956715648
2,6041,1,5.0,956715648
3,6041,2671,5.0,956715648
4,6041,356,5.0,956715648


### 4) CSR matrix 생성
`csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])`  
참고 : https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html

In [27]:
check_none = ratings.isnull().sum()  # null check -> 없음

n_user = ratings['user_id'].nunique()    # 6040
n_movie = ratings['movie_id'].nunique()  # 3628
print(type(n_user), n_user)
print(type(n_movie), n_movie)
# count내 어떤 종류의 별점이 있는지 확인
print(ratings['count'].unique())  # [5. 3. 4.]

# user_id가 같고, movie_id도 같은 데이터가 있는지 확인

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
# csr_data = csr_matrix((ratings.count, (ratings.user_id, ratings.movie_id)), shape=(n_user, n_movie))
csr_data

<class 'int'> 6040
<class 'int'> 3628
[5. 3. 4.]


<6042x3953 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
3. use_gpu : GPU를 사용할 것인지
4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [11]:
als_model = AlternatingLeastSquares(factors=300, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
#  ALS의 fit 함수는 (item, user)의 차원으로 데이터셋을 입력받으므로 현재 (plylist, songs or tags), 
# 즉 (user, item)으로 되어 있는 데이터셋을 전치주어야 합니다.
csr_data_transpose = csr_data.T

# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악

In [15]:
# movie_id로 영화 title가져오기
def get_movie_name(idx):
    if idx in movies.movie_id:
        return movies[movies['movie_id'] == idx]['title'].values[0]
    else:
        print('해당 인덱스의 영화가 존재하지 않습니다.')    

# 내 벡터와 영화 포레스트검프의 벡터가져오기
forrest_gump_id = my_movies_index[4]
my_vector, forrest_gump_vector = als_model.user_factors[my_id], als_model.item_factors[my_movies_index[4]]
# my_vector와 forrest_gump_vector를 내적하는 코드
a = np.dot(my_vector, forrest_gump_vector)

# my_vector와 father_of_the_bride_vector를 내적하는 코드
father_of_the_bride_id = 5
father_of_the_bride_vector = als_model.item_factors[father_of_the_bride_id]
b = np.dot(my_vector, father_of_the_bride_vector)

print(f'내가 선호하는 영화 {get_movie_name(forrest_gump_id)}와의 선호도 : {a}')
print(f'그 외의 영화 {get_movie_name(father_of_the_bride_id)}와의 선호도 : {b}')

내가 선호하는 영화 Forrest Gump (1994)와의 선호도 : 0.9293659925460815
그 외의 영화 Father of the Bride Part II (1995)와의 선호도 : 0.03265054151415825


### 7) 내가 좋아하는 영화와 비슷한 영화를 추천

In [16]:
def get_similar_movie(movie_title, movies, n=10):
    exist_movie = check_movie([movie_title], movies)[0]

    if exist_movie:
        movie_id = title2index([movie_title], movies)[0]
        similar_movie = als_model.similar_items(movie_id, N=n)
        similar_movie = [get_movie_name(i[0]) for i in similar_movie]
        return similar_movie
    
    print('해당 영화가 데이터에 없습니다.')
    return None

print(get_similar_movie(my_movies_title[0], movies))

['Dead Poets Society (1989)', 'Driving Miss Daisy (1989)', 'Field of Dreams (1989)', 'Karate Kid, The (1984)', 'Children of a Lesser God (1986)', 'Free Willy 3: The Rescue (1997)', 'Adventures of Elmo in Grouchland, The (1999)', 'Curse of the Puppet Master (1998)', 'Cry in the Dark, A (1988)', 'Designated Mourner, The (1997)']


### 8) 내가 가장 좋아할 만한 영화들을 추천

In [21]:
movie_recommended = als_model.recommend(my_id, csr_data, N=20, filter_already_liked_items=True)
[get_movie_name(i[0]) for i in movie_recommended]

['Toy Story 2 (1999)',
 'Jerry Maguire (1996)',
 'Rain Man (1988)',
 "You've Got Mail (1998)",
 'Field of Dreams (1989)',
 'Breakfast Club, The (1985)',
 "My Best Friend's Wedding (1997)",
 'Sleepless in Seattle (1993)',
 "Ferris Bueller's Day Off (1986)",
 "Bug's Life, A (1998)",
 'As Good As It Gets (1997)',
 'Rocky (1976)',
 'Aladdin (1992)',
 'Dogma (1999)',
 'Wedding Singer, The (1998)',
 'Driving Miss Daisy (1989)',
 'Pretty Woman (1990)',
 "Schindler's List (1993)",
 'Lion King, The (1994)',
 'L.A. Confidential (1997)']

In [22]:
# 추천 기여도 확인
recommend_movie_id = movie_recommended[0][0]
explain = als_model.explain(my_id, csr_data, itemid=recommend_movie_id)
[(get_movie_name(i[0]), i[1]) for i in explain[1]]

[('Toy Story (1995)', 0.24281749151898763),
 ('Notting Hill (1999)', 0.020565780347741307),
 ('Forrest Gump (1994)', 0.0011543239838175418),
 ('Good Will Hunting (1997)', 0.0010189472020988646),
 ('Dead Poets Society (1989)', -0.008701127979816613)]

<br><br><br><br>

## 회고
### 이번 프로젝트에서 **어려웠던 점**
```py
# 1번째 csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
csr_data = csr_matrix((ratings.count, (ratings.user_id, ratings.movie_id)), shape=(n_user, n_movie))
# TypeError: len() of unsized object

# 2번째
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape=(n_user, n_movie))
# ValueError: row index exceeds matrix dimensions

# 3번째 - 최종
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
```
csr matrix를 생성할 때 위 코드처럼 2번의 에러가 났었다.  
첫번째의 TypeError는 사실 아직도 잘 이해가 가지 않는다. 들어온 데이터가 len() 함수를 지원하지 않는다는 내용같은데, 노드를 진행할 때는 dot(.)으로 진행하였는데 잘 되었기 때문이다. 그래서 \[\]으로 변경해주니 타입에러는 없어졌다.   

두번째 ValueError는 에러코드를 보니 self.row.max()가 전달해준 shape의 0번째 인덱스보다 크다는 내용이였다. 같은 에러에 대한 질문의 댓글을 보니 영화아이디가 연속적이지 않아 unique갯수보다 더 큰 인덱스가 발생하게 된다는 댓글이 있었다. 에러내용의 max()와 얼추 맞는 듯한 내용이었다. 두가지 방법이 있었는데, movie_id를 다시 설정해주는 방법과 코드상 shape를 제거해주는 방법이 있었는데 나는 후자를 택했다.  
```py
# 2번째 에러 내용 중
if self.row.max() >= self.shape[0]:
--> 283                 raise ValueError('row index exceeds matrix dimensions')
    284             if self.col.max() >= self.shape[1]:
    285                 raise ValueError('column index exceeds matrix dimensions')
```


### 결과분석
6번 과정에서 내가 선택했던 포레스트검프 영화는 0.92로 꽤 높았고, 선택하지 않았던 'Father of the Bride Part II'영화는 0.03으로 꽤 낮게 나왔다. 내가 좋아하는 영화와 비슷한 영화나 추천해준 영화들을 보니 비슷한 장르들로 추천해주는 것을 보아 꽤 괜찮은 학습이 된 것 같다.


### 자기 다짐
위에서 났던 에러를 보니 데이터전처리가 생각보다 중요하다는 것을 깨달았다. 두가지 방법 중 시간때문에 후자를 택했지만, 전자에 대해 다시 한번 리팩토링 해봐야할 것 같다. 